In [ ]:
!pip install bitsandbytes transformers accelerate tqdm youtube-transcript-api

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,BitsAndBytesConfig
from youtube_transcript_api import YouTubeTranscriptApi
from tqdm import tqdm
import torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [ ]:
# Clear GPU memory before running
torch.cuda.empty_cache()

In [ ]:
# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load model and tokenizer
model_id = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Device set to use cuda:0


In [ ]:
def get_youtube_transcript(video_id):
    """Fetches the transcript from YouTube and returns it as a single string."""
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    text = " ".join([entry['text'] for entry in transcript])
    return text

def split_transcript(transcript, max_words=400):
    """Splits transcript into chunks ensuring sentence integrity."""
    words = transcript.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(current_chunk) >= max_words:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def summarize_transcript(transcript, max_words=400):
    """Summarizes transcript while ensuring it is treated as spoken dialogue."""
    chunks = split_transcript(transcript, max_words)
    summary = []

    for chunk in tqdm(chunks, desc="Summarizing..."):
        prompt = (
            "This is a transcript of a YouTube video. The text may have informal speech, filler words, and unstructured dialogue. "
            "Your task is to summarize it into a clear and concise format, keeping the main ideas intact.\n\n"
            f"Transcript:\n{chunk}\n\nSummary:"
        )


        response = generator(prompt, max_new_tokens=150, truncation=True)[0]['generated_text']

        summary.append(response)

    return " ".join(summary)


In [ ]:
# Example usage
video_id = "7CNElr-TAQw"  # Replace with actual video ID
transcript = get_youtube_transcript(video_id)
summary = summarize_transcript(transcript)

print("\n📝 Video Summary:\n", summary)


Summarizing...: 100%|██████████| 6/6 [00:45<00:00,  7.55s/it]


📝 Video Summary:
 This is a transcript of a YouTube video. The text may have informal speech, filler words, and unstructured dialogue. Your task is to summarize it into a clear and concise format, keeping the main ideas intact.

Transcript:
in the previous video we saw an overview of how the Transformers work we saw that there are encoders taxs here then there's the decoder Stacks here and there is also the input over here in this video we'll focus mainly on the input embedding here and how a personal encoding is added to it and fed into the [Music] network and one thing to note over here is that even though like we have all of these words which are represented as vectors in this case and unlike in RNN it is not fed one by one it is fed all together but if you have a sentence the cat is on the mat it won't be fed like the is fed first cat is fed second Etc it should all be fed together and in the case of state-of-the-art models this could be like 20,000 in length it need not be one si

In [ ]:
def summarize_all_summaries(summary_text, max_words=400):
    """Summarizes all the individual summaries into one."""
    chunks = split_transcript(summary_text, max_words)  # Reuse the split_transcript function
    final_summary = []

    for chunk in tqdm(chunks, desc="Summarizing all summaries..."):
        prompt = (
            "Here are multiple summaries of a YouTube video. "
            "Combine them into one concise and comprehensive summary that covers all the main points. "
            f"Summaries:\n{chunk}\n\nCombined Summary:"
        )

        response = generator(prompt, max_new_tokens=200, truncation=True)[0]['generated_text']
        final_summary.append(response)

    return " ".join(final_summary)

# Example usage:
final_summary = summarize_all_summaries(summary)  # Assuming 'summary' holds the combined individual summaries
print("\n📝 Final Combined Summary:\n", final_summary)

Summarizing all summaries...: 100%|██████████| 9/9 [01:11<00:00,  7.91s/it]


📝 Final Combined Summary:
 Here are multiple summaries of a YouTube video. Combine them into one concise and comprehensive summary that covers all the main points. Summaries:
This is a transcript of a YouTube video. The text may have informal speech, filler words, and unstructured dialogue. Your task is to summarize it into a clear and concise format, keeping the main ideas intact. Transcript: in the previous video we saw an overview of how the Transformers work we saw that there are encoders taxs here then there's the decoder Stacks here and there is also the input over here in this video we'll focus mainly on the input embedding here and how a personal encoding is added to it and fed into the [Music] network and one thing to note over here is that even though like we have all of these words which are represented as vectors in this case and unlike in RNN it is not fed one by one it is fed all together but if you have a sentence the cat is on the mat it won't be fed like the is fed fi

In [ ]:
import re

def extract_video_id(youtube_url):
    """Extracts the video ID from a full YouTube URL."""
    pattern = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|.*[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})"
    match = re.search(pattern, youtube_url)
    return match.group(1) if match else None

# Example usage
youtube_url = "https://www.youtube.com/watch?v=7CNElr-TAQw&t=4s&ab_channel=Vizuara"  # Example YouTube link
video_id = extract_video_id(youtube_url)

if video_id:
    transcript = get_youtube_transcript(video_id)
    summary = summarize_transcript(transcript)
    print("\n📝 Video Summary:\n", summary)
else:
    print("❌ Invalid YouTube URL. Please provide a valid link.")


Summarizing...: 100%|██████████| 6/6 [00:43<00:00,  7.21s/it]


📝 Video Summary:
 This is a transcript of a YouTube video. The text may have informal speech, filler words, and unstructured dialogue. Your task is to summarize it into a clear and concise format, keeping the main ideas intact.

Transcript:
in the previous video we saw an overview of how the Transformers work we saw that there are encoders taxs here then there's the decoder Stacks here and there is also the input over here in this video we'll focus mainly on the input embedding here and how a personal encoding is added to it and fed into the [Music] network and one thing to note over here is that even though like we have all of these words which are represented as vectors in this case and unlike in RNN it is not fed one by one it is fed all together but if you have a sentence the cat is on the mat it won't be fed like the is fed first cat is fed second Etc it should all be fed together and in the case of state-of-the-art models this could be like 20,000 in length it need not be one si